# ACV paramétrique et éco-conception : application à un drone électrique

---
Auteur :<br>
  > Félix POLLET <br>
  > felix.pollet@isae-supaero.fr
---

### Objectif du notebook
L'objectif de ce notebook est de vous donner les clés et outils pour réaliser des ACV qui vont au-delà des limites des ACV conventionnelles. Vous apprendrez notamment à implémenter des ACV paramétriques à des fins d'éco-conception.

### Cas d'étude
Le support d'étude pour ce projet est un drone multirotor électrique. Les principaux composants d'un drone (également appelé UAV pour *Unmanned Aerial Vehicle*) sont :
* Les **hélices**. Elles permettent de fournir la poussée nécessaire lors des différentes phases du vol. Les hélices sont souvent faites en matériaux composites (fibres de verre ou fibres de carbone).
* Les **moteurs** à aimants permanents. Ils convertissent la puissance électrique de la batterie (tension et courant) en puissance mécanique pour les hélices (couple et vitesse de rotation). Ils sont composés d'aimants en néodyme, de bobines en cuivre, d'un rotor et d'un stator en fer, ainsi que d'un boitier en aluminium.
* Les **batteries**. Elles stockent l'énergie nécessaire au vol sous forme chimique. Il existe différentes compositions chimiques pour les batteries. L'anode est souvent à base de graphite. La cathode est généralement composée de lithium en combinaison avec d'autres éléments, par exemple du Nickel, du Manganèse et du Cobalt (on parlera alors de batterie Li-ion *NMC*), ou encore du Fer et du Phosphate (batterie Li-ion *LFP*). Selon la chimie, la batterie pourra être plus légère (batteries NMC) ou plus "durable", au sens où sa durée de vie sera plus grande (batteries LFP).
* La **structure**. Elle se compose d'un corps central qui accueille la batterie et la charge utile, ainsi que de bras qui soutiennent les moteurs et les hélices. Pour des raisons de masse, les matériaux composites à base de carbone sont privilégiés.

Une anatomie exhaustive d'un drone multirotor peut être trouvée [ici](https://makezine.com/article/maker-news/anatomy-of-a-drone/).

Plus spécifiquement, nous allons évaluer les impacts environnementaux d'un drone utilisé pour la livraison de petits colis.

L'unité fonctionnelle est la suivante :

> **Unité fonctionnelle** :
>
> <cite>Réaliser n fois la livraison d'un colis de 1 kg à une distance de 10 km.</cite>

Dans un premier temps, on prendra $n=2500$. Cependant ce choix pourra faire l'objet d'une étude paramétrique.

<p align="center" width="100%">
    <img width="33%" src="./figures/drone_delivery.jpg">
</p>

*Note: Ce notebook présente avant tout une méthodologie. On ne présuppose pas ici de l'utilité sociale de la livraison de colis, ni de l'intérêt d'utiliser des drones pour cette application. Par ailleurs, les compétences acquises ici peuvent tout à fait servir de base à une étude plus approfondie sur l'intérêt environnemental des drones vis-à-vis d'autres moyens de transport.*

### Déroulé du notebook
Le déroulé du notebook est le suivant :
1. [Création d'un projet ACV en Python](#section_1)
2. [Création des processus et des paramètres ACV pour notre étude de cas](#section_2)
3. [Etude ACV "classique" pour évaluer les impacts d'un drone donné](#section_3)
4. [Eco-conception : intégration de l'ACV dans un problème d'optimisation](#section_4)


<a id='section_1'></a>
## 1. Création du projet ACV

Nous allons utiliser dans ce notebook deux librairies Python **open-source** pour réaliser des ACV paramétriques:

* [Brightway2](https://documentation.brightway.dev/en/latest/index.html) permet de faire des ACV avancées avec des temps de calcul réduits.
* [LCA Algebraic](https://github.com/oie-mines-paristech/lca_algebraic) est une surcouche de brightway2 qui permet de paramétriser efficacement les ACV.

Avant toute chose, nous devons importer ces librairies et créer un projet pour notre cas d'étude. C'est l'objectif de cette section.

In [ ]:
# Importation des librairies
import brightway2 as bw
from lca_algebraic import *
from lca_algebraic.stats import *
import os.path as pth
import sys
from lca_course.helper_functions import create_lca_project, recursive_activities, graph_activities
import lca_course
from pathlib import Path
path = Path(lca_course.helper_functions.__file__).parent.absolute()
DB_PATH = pth.join(path, r"background_db/datasets")  # linux
# DB_PATH = pth.join(path, r"background_db/datasets")  # windows
BACKGROUND_DB = "ecoinvent db"

La création du projet consiste principalement à importer une base de données ACV qui servira de base à nos calculs. L'importation peut prendre quelques minutes, patience...

In [ ]:
# Creation du projet ACV
PROJECT_NAME = "certificat_2023"  # le nom de votre projet
create_lca_project(PROJECT_NAME, DB_PATH, BACKGROUND_DB)

**Background vs Foreground** <br>
On distingue en ACV les processus *background* (arrière-plan) et *foreground* (premier-plan). 
* Le *background* regroupe les processus sur lesquels le décideur (nous) n'a pas d'influence directe. Par exemple, les procédés de fabrication d'un moteur électrique sont en dehors de notre champ d'action si nous ne sommes pas constructeur de moteurs. 
* Par opposition, le *foreground* regroupe les processus qui sont sous notre contrôle. Si notre corps de métier est de concevoir des drones à partir de composants du marché, le nombre et la taille des moteurs que nous achetons pour fabriquer les drones relèvent de notre champ d'action. Les processus *foreground* se construisent généralement par-dessus les processus *background*.

Ainsi, la base de données importée ci-dessus est considérée comme une base de donnée *background*. Elle est figée. En complément, nous allons construire une base de données *foreground* qui regroupe l'ensemble des processus qui touchent directement à la conception et à l'utilisation du drone (quantité de moteurs, de batteries, consommation d'électricité en vol, etc).

![bdd](./figures/background_foreground.png)

Les quelques lignes de commande ci-dessous permettent d'initialiser la base de données *foreground*.

In [ ]:
# Nous utilisons une base de données séparées pour définir les processus de "premier plan" (foreground)
USER_DB = 'foreground db'

# Remise à zéro de la base de données foreground
resetDb(USER_DB)
resetParams()

On peut désormais vérifier la liste des bases de données présentes dans notre project:

In [ ]:
# Apercu des bases de données
list_databases()

Ici, *biosphere3* est une autre base de données de type *background*, qui ne contient que les échanges avec l'environnement (matières premières, émissions dans l'air et dans les sols, etc). Elle est complémentaire avec EcoInvent, cette dernière ne contenant que les processus de la "technosphère".

<a id='section_2'></a>
## 2. Création des processus et des paramètres ACV

L'objectif de cette section est de 
* définir les **processus** permettant de satisfaire la fonction du système drone, 
* et de créer les **paramètres** caractérisant les flux de produits. Les flux correspondent aux quantités de produits nécessaires et achetées pour remplir cette fonction.

Afin de se faciliter la tâche, nous allons construire un arbre des processus dans lequel on fera apparaître les processus et les flux de produits pour réaliser la fonction "transport d'une charge utile par drone". Les flux seront exprimés par unité fonctionnelle.

Voici un exemple d'arbre des processus pour l'éclairage par ampoules à incandescence :

![arbre](./figures/arbre_processus_eclairage.jpg)

Dans cet exemple, les quantités échangées sont fixes. **Tout l'intérêt de l'ACV paramétrique est de pouvoir faire varier ces quantités en les associant à des paramètres** (dans la mesure où ces quantités sont liées au *foreground*). Ainsi, on pourra rapidement explorer une multitude de scénarios. Par exemple, pour satisfaire la même unité fonctionnelle, que se passe-t-il si j'achète 1 ampoule fluorescente qui consommera 66 kWh plutôt que 6 ampoules qui consommeront 360 kWh ? Et si je change le mix électrique ? Et si j'achète mes ampoules plus proches ?

De la même manière, il existe une multitude de drones qui peuvent répondre à la fonction "2500 livraisons d'un colis de 1 kg", et tous ne nécessiteront pas la même quantité de batteries ni ne consommeront la même quantité d'énergie en vol.

***Exercice préliminaire**: sur papier, réaliser l'arbre des processus pour le transport de colis avec un drone. On fera notamment apparaître différents mix électriques (Français, Européen et Américain par exemple) et différents types de batterie (NMC ou LFP).*

### 2.1) Création des paramètres

La librairie Python LCA algebraic dispose de fonctions permettant de créer des paramètres ACV. Deux types de paramètres peuvent être implémentés :
* `FloatParam` pour les paramètres qui prennent des valeurs numériques. Exemple: la quantité d'énergie consommée pour satisfaire l'unité fonctionnelle.
* `EnumParam` pour les paramètres qui prennent des valeurs discrètes dans une liste prédéfinies de choix. Exemple: le choix d'une région pour le mix électrique (France, Etats-Unis, Europe).

***Exercice de programmation**: implémenter les paramètres identifiés dans l'arbre des processus.*

In [ ]:
### Définition des paramètres

# Energie consommée sur une mission
mission_energy = newFloatParam(  # ce paramètre est un nombre (float)
    'mission_energy',  # nom du paramètre
    default=1.0,  # valeur par défaut
    unit='kWh',  # unité
    min=0, max=1000,  # bornes sur ce paramètre (non utilisé ici)
    description="energy consumption for a single mission",  # description
    dbname=USER_DB  # on enregistre le paramètre dans notre base de données foreground
)

# Mix électrique
elec_switch_param = newEnumParam(  # ce paramètre est un switch
    'elec_switch_param',
    values=['us', 'eu', 'fr'], # valeurs possibles de ce paramètre
    default='eu',
    description="switch on electricty mix",
    dbname=USER_DB
)

# Durée de vie du drone, en nombre de missions réalisées --> pour calculer l'énergie totale consommée sur la durée de vie du drone
n_cycles = ...

# Masse des hélices
mass_propellers = ...

# Masse de moteurs
mass_motors = ...

# Masse de la structure
mass_structure = ...

# Masse de la batterie
mass_batteries = ...

# Durée de vie de la batterie, en nombre de cycles
n_cycles_battery = ...

# Types de batterie
battery_type = newEnumParam(
    'battery_type',
    values=["nmc", "lfp"],  # valeurs possibles de ce paramètre (différentes technologies de batteries)
    default="nmc",
    description="battery technology",
    dbname=USER_DB
)

Une fois les paramètres ACV déclarés, on peut les afficher avec la commande suivante :

In [ ]:
# Liste des paramètres
list_parameters()

### 2.2) Définition des processus

A partir de l'arbre des processus, on a vu que les processus *foreground* échangent des quantités avec les processus *background*. Avant toute chose, il s'agit donc d'aller récupérer les processus *background* qui nous intéressent dans la base de données importée.

**Récupération des processus background** <br>
Les quelques lignes de code ci-dessous permettent de rechercher un processus (également appelé "activité" en langage brightway2) à partir de mots clés, et de récupérer le code/identifiant correspondant.

In [ ]:
### Recherche d'un processus/activité dans la base de données background

# Base de données dans laquelle on recherche
db = bw.Database(BACKGROUND_DB)

# Recherche par mots clés
activities = [
    act for act in db if 
    'market for battery' in act['name']
    and 'cell' in act['name']
    # and 'NMC811' in act['name']
    # ...
]

# Affichage des processus contenant le mot clé, et du code/identifiant associé
for act in activities:
    print(act, act.key) 

Lorsqu'on connaît le code d'un processus, on peut sélectionner directement ce processus avec la fonction `findActivity`.

***Exercice de programmation**: importer les processus background identifiés dans l'arbre des processus.*

In [ ]:
# Batterie Li-ion NMC
battery_nmc = findActivity(
    db_name=BACKGROUND_DB,
    code='52e3cdd70890530eada4fbcef2741406'
)

# Batterie Li-ion LFP
battery_lfp = findActivity(
    db_name=BACKGROUND_DB,
    code='f6036ad86fb205d8712754f2fac10a16'
)

# Matériau composite pour l'hélice et la structure
composite = ...

**Création des processus foreground**<br>
On peut désormais créer nos processus *foreground* à partir des processus *background* et des paramètres ACV caractérisant les flux de produits.<br>
Pour cela, deux fonctions sont offertes par la librairie LCA algebraic :
* `newActivity`, fonction de base pour créer un processus
* `newSwitchAct`, qui permet de créer un "switch" virtuel pour sélectionner un processus parmi plusieurs choix possibles. Exemple: choix du mix électrique.

In [ ]:
from sympy import ceiling

### NIVEAU -2 ###
# Création du processus "battery" pour sélectionner le type de batterie et le nombre de batteries nécessaires au cours de la vie du drone
battery = newSwitchAct(
    USER_DB,    # on enregistre le processus dans notre base de données foreground 
    "battery",  # nom du processus
    battery_type,  # paramètre de switch précedemment défini
    { # processus à sélectionner selon de la valeur du switch
        "nmc": battery_nmc,
        "lfp": battery_lfp,
    }
)

# Création du processus "propeller" pour définir le type d'hélice du drone
propeller = newActivity(
    USER_DB,
    "propeller",
    "kg",  # unité
    exchanges={  # échanges avec les autres processus, sous la forme (processus : quantité fixe ou définie à partir d'une formule)
        composite: 1.0,  # pour 1 kg d'hélices on a besoin de 1 kg de composite
        # composite: 0.8  # on peut également définir une valeur inférieure à 1.0 pour prendre en compte le recyclage
    }
)

# Création du processus "motor"
motor = ...

# Création du processus "structures"
structure = ...

### NIVEAU -1 ###
# Création du processus "production d'un drone"
production = newActivity(
    USER_DB, 
    "production", 
    "uav", # Unité
    exchanges= { # échanges avec les autres processus, sous la forme (processus : quantité définie à partir d'une formule)
        structure: mass_structure, 
        propeller: mass_propellers,
        motor: mass_motors,
        battery: mass_batteries * ceiling(n_cycles / n_cycles_battery),  # on référence le processus "battery" que l'on vient de créer
    })


# Création du processus "operation" pour la consommation d'énergie en vol.
operation = newSwitchAct(
    USER_DB, 
    "operation",
    elec_switch_param, # paramètre de switch
    {
        "us" : electricity_us,
        "eu" : electricity_eu,
        "fr" : electricity_fr,
    })


### NIVEAU 0 ###
# Dernière pièce du puzzle: création du processus principal permettant de fournir une unité fonctionnelle
model = newActivity(
    USER_DB,
    "uav_delivery",
    "functional unit",
    exchanges= {
        production: 1.0,  # on a besoin de 1 drone...
        operation: n_cycles * mission_energy,  # ...et d'une certaine quantité d'électricité
    }
)

**Visualisation de l'arbre des processus**<br>
A des fins de vérifications, on peut afficher l'arbre des processus créé à partir de nos activités et paramètres :

In [ ]:
NETWORK_PATH = './lca_activities.html'
graph_activities(USER_DB, model, NETWORK_PATH)

from IPython.display import IFrame
IFrame(src='lca_activities.html', width="100%", height="500px")

<a id='section_3'></a>
## 3. Une première ACV (enfin !)

Ca y est, nous avons notre modèle ACV ! Maintenant, place à l'évaluation des impacts et aux interprétations.<br>
La démarche, très simple, est la suivante :
1) Fixer les valeurs des paramètres
2) Indiquer les méthodes d'évaluation d'impact que l'on souhaite utiliser
3) Lancer le calcul

Dans un premier temps, on va chercher à évaluer les impacts environnementaux d'un drone fictif dont les caractéristiques principales sont données ci-dessous :


| Caractéristiques du drone      |        |
|-----------------------|--------|
| Nombre d'hélices      | 4      |
| Charge utile max.     | 1.5 kg   |
| Endurance max.        | 30 min |
| Masse max.            | 2.6 kg     |
| Masse des hélices     | 0.06 kg      |
| Masse des moteurs     | 0.17 kg      |
| Masse des batteries   | 0.59 kg       |
| Type de batterie      | NMC (500 cycles max.)  |
| Masse de la structure | 0.27  kg     |


Les performances de ce drone pour notre mission de référence sont également fournies :

| Performances pour la mission de référence   |        |
|-----------------------|--------|
| Charge utile          | 1 kg   |
| Durée du vol          | 25 min |
| Energie consommée     | 110  Wh  |

***Exercice de programmation**: renseigner les valeurs des paramètres ACV puis lancer l'évaluation des impacts.*

In [ ]:
### Valeurs des paramètres pour l'ACV
parameters = dict(
    elec_switch_param="eu",  # "fr", "eu" ou "us"
    mission_energy=0.110,    # [kWh] attention à l'unité !
    n_cycles=2500,
    mass_batteries=0.59,     # [kg]
    # ...
)

In [ ]:
### Impacts à évaluer. Une liste des méthodes disponibles se trouve dans le fichier "lca_methods.csv"
impacts = [
    ('ReCiPe 2016 v1.03, midpoint (H) no LT', 'climate change no LT', 'global warming potential (GWP1000) no LT'),
    ('ReCiPe 2016 v1.03, midpoint (H) no LT', 'acidification: terrestrial no LT', 'terrestrial acidification potential (TAP) no LT'),
    ('ReCiPe 2016 v1.03, midpoint (H) no LT', 'water use no LT', 'water consumption potential (WCP) no LT'),
    ('ReCiPe 2016 v1.03, midpoint (H) no LT', 'material resources: metals/minerals no LT', 'surplus ore potential (SOP) no LT'),
]

In [ ]:
### Calcul de l'inventaire + évaluation des impacts
results = multiLCAAlgebric(
    model, # le modèle à évaluer 
    impacts, # les catégories d'impacts à évaluer
    **parameters # paramètres de l'ACV
)
results

**Contributions aux impacts**<br>
Il est également possible d'obtenir les contributions des différents processus :

In [ ]:
### Contributions aux impacts

# Processus à évaluer
#activities = [production, operation]
activities = [propeller, motor, structure, battery, operation]

# Calcul des contributions
results = pd.DataFrame()
for act in activities:
    res = multiLCAAlgebric(
        model,
        impacts,
        **parameters,

        extract_activities=[act]
    )
    res.index.values[0] = act['name']
    results = pd.concat([results, res])
    
# Affichage des résultats sous forme d'un tableau
results

In [ ]:
# Affichage des résultats sous forme d'un histogramme
results.transpose().plot.bar(xlabel='Impact category', ylabel='Impact score', stacked=True)
plt.xticks(rotation=90);
plt.legend(loc='lower center', 
           bbox_to_anchor=(0.5, 1.05),
           fancybox=True, shadow=True);

Intéressant, mais les différents ordre de grandeurs des impacts ne rendent pas la lecture très simple. Le graphe ci-dessous donne un meilleur aperçu des contributions :

In [ ]:
# Affichage des contributions
import matplotlib.ticker as mtick

relative_results = results/results.sum()
ax = relative_results.transpose().plot.bar(xlabel='Impact category', ylabel='Contribution', stacked=True)
plt.xticks(rotation=90);
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.legend(loc='lower center', 
           bbox_to_anchor=(0.5, 1.05),
           fancybox=True, shadow=True);

**Analyse de sensibilité**<br>
Finalement, on peut s'intéresser à l'influence de la durée de vie du drone. Dans le cas de référence, la durée de vie considérée est de 2500 missions. Si cette durée diminue, la contribution de la phase d'opération (utilisation d'électricité) diminue par rapport aux phases de fabrication.

In [ ]:
### Evolution des impacts avec la durée de vie du drone (nombre de missions)

# Choix des paramètres
parameters["n_cycles"] = list(np.arange(100.0, 10000.0, 10))  # l'analyse des impacts sera faite pour l'ensemble de ces valeurs
parameters["elec_switch_param"] = "eu"  # "fr", "eu" ou "us"

# Choix de l'impact à évaluer
impacts = [
    ('ReCiPe 2016 v1.03, midpoint (H) no LT', 'climate change no LT', 'global warming potential (GWP1000) no LT'),
]

# Processus à évaluer
#activities = [production, operation]
activities = [propeller, motor, structure, battery, operation]

# Evaluation des impacts
results = pd.DataFrame()
for act in activities:
    res = multiLCAAlgebric(
        model, # le modèle à évaluer 
        impacts, # les catégories d'impacts à évaluer
        **parameters, # paramètres de l'ACV
        
        extract_activities=[act]
    )
    res.columns=[act.as_dict()['name']]
    results = pd.concat([results, res], axis=1)

# Affichage des résultats
results['n_cycles']=parameters['n_cycles']
ax = results.plot.area(x='n_cycles', xlabel='number of cycles', ylabel='Impact score')
plt.show()

**Conclusion intermédiaire**<br>
Les premières sections de ce notebook ont permis de mettre en place un modèle d'ACV paramétrique, procurant ainsi une grande flexibilité pour l'évaluation de différents scénarios. Une première application a permis d'évaluer les impacts environnementaux d'un drone, et de comparer ces impacts selon plusieurs hypothèses sur la durée de vie du drone ou encore le mix électrique.

Cependant, un unique drone a été étudié. Mais ce n'est pas le seul drone à pouvoir accomplir la fonction demandée. Ce n'est peut être pas non plus le drone optimal pour cette fonction. La suite de ce notebook s'intéresse donc à faire **varier les paramètres relatifs au design du drone** (masse des composants, énergie consommée), qui jusqu'ici étaient fixés. L'objectif est de trouver le drone dont l'impact environnemental est minimal. On entre ainsi dans une démarche d'**éco-conception**.

<a id='section_4'></a>
## 4. De l'ACV paramétrique à l'éco-conception

L'éco-conception est un processus itératif : à partir d'une évaluation environnementale initiale, on recherche des stratégies d'amélioration qu'on implémente dans une nouvelle conception. Une analyse comparative est alors effectuée, et le processus est répété jusqu'à obtenir la meilleure conception du produit. Deux écueils sont souvent rencontrés dans cette démarche :
1. l'ACV intervient parfois trop tard dans la conception du produit, la marge de manoeuvre étant alors limitée.
2. Le processus itératif peut être très chronophage s'il est réalisé "à la main".

Pour palier à ces problèmes, nous proposons ici d'adopter une approche d'**optimisation multidisciplinaire de la conception**. Détaillons un peu ces termes :

* **Optimisation**: on parle de problème d'optimisation lorsqu'on cherche une solution qui maximise ou minimise une *fonction objectif*, par exemple l'impact environnemental. L'exploration des solutions alternatives se fait en modifiant la valeur de *variables de conception* telles que le diamètre des hélices et la taille de la batterie. Des *contraintes* sont ajoutées au problème pour assurer les performances et la faisabilité technique du produit. Un algorithme d'optimisation est souvent employé pour faire ce travail d'exploration à notre place. Mathématiquement, un problème d'optimisation s'exprime de la manière suivante :

<math>\begin{align}
&{\operatorname{minimize}}& & f(x) \\
&\operatorname{subject\;to}
& &h_i(x) = 0, \quad i = 1, \dots,m_1 \\ 
&&&g_j(x) \leq 0, \quad j = 1,\dots,m_2 \\
&\operatorname{and}
& &x \in X \subseteq R^n
\end{align}</math>


* **Multidisciplinaire**: l’éco-conception recherche le juste compromis de l’environnement avec toutes les autres contraintes liées à la conception du produit. Ainsi, elle intègre les réflexions environnementales au même titre que la faisabilité technique, les performances économiques, etc. Dans le cas du drone, l'évaluation environnementale cohabite avec des analyses aérodynamiques, electro-mécaniques et structurelles.

* **Conception**: la conception d'un produit, et plus particulièrement d'un système aéronautique, repose sur trois piliers :
    * Les performances attendues de ce produit pour répondre à un cahier des charges donné. Par exemple, le temps de vol minimal pour un drone.
    * Les technologies utilisées dans la conception du produit. Par exemple, le type de batterie utilisé.
    * Le bon dimensionnement de ce produit. Un drone de transport de passagers sera évidemment surdimensionné pour livrer un petit colis. Ce dernier point est étroitement lié au concept d'optimisation, via les variables de conceptions.

L'objectif de ce notebook n'est pas de rentrer dans les détails de la conception préliminaire d'un drone. Le schéma ci-dessous résume la procédure d'optimisation multidisciplinaire de la conception que nous adoptons dans la suite.

<div style="text-align:center"><img src="./figures/design_optimization_2.png" /></div>

Les grandeurs de référence pour le chahier des charges en performances et les hypothèses technologiques sont les suivantes:

| Cahier des charges performances |        |
|---------------------------------|--------|
| Charge utile                    | 1 kg   |
| Durée du vol                    | 25 min |


|  Technologies |                                 |                                   |
|---------------|---------------------------------|-----------------------------------|
| Hélices       | Ratio masse / diamètre | 0.68 kg/m$^3$                     |
| Moteurs       | Ratio masse / couple            | 0.28 kg/(N.m)$^{3/3.5}$           |
| Batterie      | Energie spécifique              | 200 Wh/kg                         |
|               | Puissance spécifique            | 5000 W/kg                         |
| Structure     | Densité matériau                | 1700 kg/m$^3$                     |


Enfin, les variables de conception sont les suivantes :

| Variables de conception |                   |
|-------------------------|-------------------|
| Hélices                 | Diamètre          |
| Moteurs                 | Couple nominal    |
| Batterie                | Energie stockée   |
| Structure               | Longueur des bras |

### 4.1) Code de dimensionnement et d'évaluation

Le code ci-dessous permet, à partir des exigences de performances, des hypothèses technologiques et des variables de conception, de dimensionner et évaluer un drone multirotor.

C'est une version volontairement simplifiée :
1. Seuls les hélices, les moteurs, la batterie et la structure sont modélisés.
2. Les exigences en performance se cantonnent au vol stationnaire. Il s'agit alors de s'assurer que la chaîne propulsive est capable de fournir suffisamment de poussée pour contrebalancer la gravité. Les scénarios plus complexes, comme le décollage et la croisière, ne sont pas considérés.
3. Les caractéristiques des composants sont obtenues à partir de modèles simplistes. Typiquement, les masses sont calculées à partir de ratios représentatifs.

In [ ]:
import numpy as np

def drone_sizing(performances, technologies, options_acv, variables_conception):
    """
    Modèle pour évaluer le dimensionnement du drone et de ses composants. 
    On se place dans le cas du vol stationnaire. 
    Il faut alors dimensionner les composants pour fournir suffisament de puissance (contrebalancer le poids), suffisament longtemps (durée de la mission).
    Ce modèle doit être appelé dans un problème d'optimisation pour assurer le respect des contraintes et la minimisation de l'objectif (masse ou impact environnemental).
    """
    
    ### Performances requises
    M_load = performances[0]  # [kg] charge à transporter
    t_mission = performances[1]  # [min] durée du vol requise
    
    ### Technologies de références
    propeller_mass_diameter_ratio = technologies[0]  # [kg/m^3] paramètre pour relier la masse d'une hélice à son diamètre
    motor_mass_torque_ratio = technologies[1]  # [kg/(N.m)^(3/3.5)] paramètre pour relier la masse d'un moteur à son couple nominal
    battery_specific_energy = technologies[2]  # [Wh/kg] rapport énergie/masse de la batterie
    battery_specific_power = technologies[3]  # [W/kg] rapport puissance/masse de la batterie
    material_density = technologies[4]  # [kg/m3] densité du matériau pour la structure
    
    
    ### Dimensionnement des composants à partir des variables de conception
    # Hélices
    D_propeller = variables_conception[0] # [m] diamètre hélices
    N_propeller = 4  # [-] nombre d'hélices
    M_propeller = N_propeller * propeller_mass_diameter_ratio * D_propeller ** 3  # [kg] estimation de la masse des hélices
    
    # Moteurs
    T_motor = variables_conception[1] # [N.m] couple nominal que peuvent fournir les moteurs
    N_motor = N_propeller  # il y a un moteur par hélice
    M_motor = N_motor * motor_mass_torque_ratio * T_motor ** (3/3.5)  # [kg] estimation de la masse des moteurs
    
    # Batterie
    E_battery = variables_conception[2]  # [Wh] energie contenue dans la batterie
    M_battery = E_battery / battery_specific_energy  # [kg] estimation de la masse de la batterie
    P_battery = M_battery * battery_specific_power  # [W] estimation de la puissance de la batterie
    
    # Structure
    L_arm = variables_conception[3]  # [m] longueur des bras
    N_arm = N_propeller  # [-] un bras par hélice
    D_in = 0.05  # [m] diamètre interne des bras (tubes)
    D_out = 0.051  # [m] diamètre externe des bras (tubes)
    M_arm = N_arm * np.pi / 4 * (D_out**2 - D_in**2) * L_arm * material_density  # [kg] masse des bras
    M_body = 1.5 * M_arm  # [kg] estimation de la masse du corps central
    M_structure = M_arm + M_body  # [kg] masse de structure
    
    # Masse totale
    M_drone = M_load + M_propeller +  M_motor + M_battery + M_structure
    
    
    ### Evaluation des performances en vol stationnaire
    # Hélices
    rho_air = 1.225  # [kg/m^3] densité de l'air
    Ct = 0.1  # [-] coefficient de poussée
    Cp = 0.04  # [-] coefficient de puissance
    F_propeller = M_drone * 9.81 / N_propeller  # [N] poussée à fournir (par hélice)
    n_propeller = (F_propeller / (rho_air * Ct * D_propeller**4)) ** (1/2)  # [Hz] vitesse de rotation de l'hélice
    P_propeller = rho_air * Cp * n_propeller**3 * D_propeller**5  # [W] puissance de l'hélice
    T_propeller = P_propeller / (2 * np.pi * n_propeller)  # [N.m] couple minimal que le moteur doit être capable de fournir à l'hélice
    
    # Moteurs
    n_motor = n_propeller  # le moteur tourne à la même vitesse que l'hélice
    eta_motor = 0.9  # [-] rendement du moteur (considéré constant ici)
    P_motor = T_motor * (2 * np.pi * n_motor) * eta_motor  # [W] puissance électrique que la batterie doit être capable de fournir au moteur
    
    # Energie consommée sur la mission
    E_mission = N_motor * P_motor * (t_mission / 60)  # [Wh] énergie requise pour compléter la mission
    
    
    ### ACV
    impacts = options_acv["impact"]
    lca_parameters = dict(
        n_cycles=options_acv["n_cycles"],
        elec_switch_param=options_acv["elec_switch_param"],        
        mass_propellers=M_propeller,
        mass_motors=M_motor,
        mass_structure=M_structure,
        mass_batteries=M_battery,
        battery_type=options_acv["battery_type"],
        n_cycles_battery=options_acv["n_cycles_battery"],
        mission_energy=E_mission/1000,  # [kWh] --> diviser par 1000
    )
    results = multiLCAAlgebric(
        model,
        impacts,
        **lca_parameters
    )
    lca_score = results.iloc[0].values[0]
    
    
    ### Contraintes et objectif d'optimisation
    # Contraintes
    cnstr_couple = (T_motor - T_propeller) / T_motor  # contrainte pour s'assurer que les moteurs peuvent fournir le couple nécessaire
    cnstr_puissance = (P_battery - N_motor * P_motor) / P_battery  # contrainte pour s'assurer que la batterie est suffisant puissante
    cnstr_bras = (L_arm - (D_propeller / 2) / (np.sin(np.pi / N_arm))) / L_arm  # contrainte sur la longueur des bras pour que les hélices ne se touchent pas
    cnstr_energie = (E_battery - E_mission) / E_battery  # contrainte pour s'assurer que la batterie a suffisant d'énergie pour conclure la mission
    constraints = [cnstr_couple, cnstr_puissance, cnstr_bras, cnstr_energie]
    
    # Fonction objectif à minimiser
    obj_mass = M_drone  # minimisation de la masse du drone
    objc_energy = E_mission  # minimisation de l'énergie consommée
    obj_lca = lca_score  # minimisation de l'impact environnemental
    objectives = [obj_mass, objc_energy, obj_lca]
    
    return constraints, objectives, lca_parameters

### 4.2) Performances et technologies (cahier des charges)

In [ ]:
### Performances souhaitées
performances = [
    # ...,  # [kg] masse de charge utile à transporter
    # ..., # [min] durée du vol
]

### Technologies de référence
technologies = [
    # ...,    # [kg/m^3] paramètre pour relier la masse d'une hélice à son diamètre
    # ...,    # [kg/(N.m)^(3/3.5)] paramètre pour relier la masse d'un moteur à son couple nominal
    # ...,   # [Wh/kg] rapport énergie/masse de la batterie
    # ...,  # [W/kg] rapport puissance/masse de la batterie
    # ...,  # [kg/m3] densité du matériau pour la structure
]

### Options ACV qui ne sont pas liées directement au dimensionnement
options_acv = {
    "impact": [('ReCiPe 2016 v1.03, midpoint (H) no LT', 'climate change no LT', 'global warming potential (GWP1000) no LT')],
    "elec_switch_param": "fr",
    "n_cycles": 2500,
    "n_cycles_battery": 500,
    "battery_type": "nmc",
}

### 4.3) Problème d'optimisation

Le processus itératif pour explorer l'espace des solutions et trouver le design optimal est réalisé en s'appuyant sur un algorithme d'optimisation. Nous utilisons ici *SLSQP*, un algorithme de [descente de gradient](https://fr.wikipedia.org/wiki/Algorithme_du_gradient). Cet algorithme nécessite un point de départ pour effectuer ses recherches. Des bornes sont également définies sur les variables de conception afin de réduire l'espace de recherche.

In [ ]:
from scipy.optimize import fmin_slsqp

### Problème d'optimisation
# Point de départ pour les variables de conception
initial_guess = [
    0.1,    # [m] D_propeller
    0.01,   # [N.m] T_motor
    10.0,   # [Wh] E_battery
    0.1,    # [m] L_arm
]

# Bornes sur les variables de conception
bounds = [
    (0.1, 1.0),     # [m] D_propeller
    (0.01, 10.0),   # [N.m] T_motor
    (10.0, 1000.0), # [Wh] E_battery
    (0.1, 2.0),     # [m] L_arm
]

# Contraintes d'optimisation
contraintes = lambda x: drone_sizing(performances, technologies, options_acv, x)[0]

# Objectif d'optimisation
objectif_mass = lambda x: drone_sizing(performances, technologies, options_acv, x)[1][0]
objectif_energy = lambda x: drone_sizing(performances, technologies, options_acv, x)[1][1]
objectif_environmental = lambda x: drone_sizing(performances, technologies, options_acv, x)[1][2]

### 4.4) Résultats

Nous pouvons désormais faire tourner l'optimisation ! Cela peut prendre quelques secondes...

In [ ]:
# Routine d'optimisation
variables_conception_opt = fmin_slsqp(  # SLSQP est un algorithme d'optimisation à descente de gradient
    func=objectif_environmental,  # fonction objectif à minimiser
    x0=initial_guess,  # point de départ de l'algorithme
    bounds=bounds,  # bornes sur les variables de conception
    f_ieqcons=contraintes,  # contraintes d'optimisation
    iter=1500,  # nombre max d'itérations
    acc=1e-6,  # critère d'arrêt de l'optimisation (précision)
)

# Affichage sommaire des résultats
print("Variables de conception optimales :", variables_conception_opt)
print("Masse totale à l'optimum :", drone_sizing(performances, technologies, options_acv, variables_conception_opt)[1][0])
print("Score environnemental à l'optimum :", drone_sizing(performances, technologies, options_acv, variables_conception_opt)[1][2])

On peut alors afficher les résultats détaillés de l'ACV pour le drone optimal. Pour cela on récupère d'abord les paramètres de l'ACV :

In [ ]:
# Paramètres ACV du drone optimal
lca_parameters = drone_sizing(performances, technologies, options_acv, variables_conception_opt)[2]

# Petite astuce d'affichage
pd.DataFrame.from_dict(lca_parameters, orient='index', columns=['value'])

Puis on procède comme tout à l'heure :

In [ ]:
### Contributions aux impacts

# Processus à évaluer
activities = [propeller, motor, structure, battery, operation]

# Calcul des contributions
lca_results = pd.DataFrame()
for act in activities:
    res = multiLCAAlgebric(
        model,
        impacts,
        **lca_parameters,

        extract_activities=[act]
    )
    res.index.values[0] = act['name']
    lca_results = pd.concat([lca_results, res])

# Affichage des résultats sous forme d'un histogramme
lca_results.transpose().plot.bar(xlabel='Impact category', ylabel='Impact score', stacked=True)
plt.xticks(rotation=90);
plt.legend(loc='lower center', 
           bbox_to_anchor=(0.5, 1.05),
           fancybox=True, shadow=True);

**Optimum(s) et objectif(s) de minimisation**<br>
Pour un même cahier des charges, comparer les designs de drones obtenus pour différents objectifs de minimisation: masse, énergie et impact environnemental.<br>
Qu'observez vous ?

In [ ]:
# Routine d'optimisation
variables_conception_opt = fmin_slsqp(  # SLSQP est un algorithme d'optimisation à descente de gradient
    func=objectif_mass,  # fonction objectif à minimiser
    x0=initial_guess,  # point de départ de l'algorithme
    bounds=bounds,  # bornes sur les variables de conception
    f_ieqcons=contraintes,  # contraintes d'optimisation
    iter=1500,  # nombre max d'itérations
    acc=1e-6,  # critère d'arrêt de l'optimisation (précision)
)

# Affichage sommaire des résultats
print("Variables de conception optimales :", variables_conception_opt)
print("Masse totale à l'optimum :", drone_sizing(performances, technologies, options_acv, variables_conception_opt)[1][0])
print("Score environnemental à l'optimum :", drone_sizing(performances, technologies, options_acv, variables_conception_opt)[1][2])

**Sensibilité au choix technologique: batterie low-tech versus batterie high-tech**

On cherche désormais à comparer deux designs de drones optimisés pour des technologies de batteries différentes dont les caractéristiques sont données ici :

| Type de batterie | Energie spécifique | Nombre de cycles max. |
|------------------|--------------------|------------------|
| Li-Ion NMC       | 200 Wh/kg          | 500              |
| Li-Ion LFP       | 130 Wh/kg          | 2000             |


Modifier les grandeurs associées dans la liste des paramètres technologiques et dans les paramètres ACV de la fonction `drone_sizing`. Faire tourner à nouveau l'optimisation et comparer les impacts environnementaux des deux drones. <br>
Quelle technologie de batterie permet de réduire au plus les impacts environnementaux choisis ? Vérifiez vos résultats pour plusieurs mix électriques et durées de vie du drone. Vous pouvez également étudier d'autres impacts environnementaux.

In [ ]:
# ...